# Classification supervisée basée sur les clusters

**Objectif :** Développer un modèle de classification supervisée pour prédire les phénotypes métaboliques du diabète en utilisant les résultats du clustering K-means.

---

## Vue d'ensemble du pipeline

Cette analyse comprend les étapes suivantes :
1. **Importation des bibliothèques** et configuration
2. **Chargement et préparation des données**
3. **Preprocessing** (standardisation et équilibrage des classes)
4. **Initialisation des modèles** de machine learning
5. **Entraînement et évaluation initiale**
6. **Validation croisée** pour robustesse
7. **Optimisation des hyperparamètres** avec GridSearchCV
8. **Évaluation finale** et sélection du meilleur modèle
9. **Sauvegarde** du modèle optimal

## 1. Importation des bibliothèques

Importation de toutes les bibliothèques nécessaires pour l'analyse de classification supervisée.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import RandomOverSampler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
import joblib

## 2. Chargement et préparation des données

Chargement des datasets issus du clustering et définition des variables explicatives et de la variable cible.

In [2]:
# --- Chargement et préparation des données ---

df_cluster_final = pd.read_csv('../data/df_cluster_final.csv')
df_cluster = pd.read_csv('../data/df_cluster.csv')

# Cible
y = df_cluster_final['cluster']

# Variables sélectionnées
X = df_cluster[['Glucose', 'BMI', 'Age', 'DiabetesPedigreeFunction']]

## 3. Preprocessing des données

### 3.1 Division train/test et standardisation

In [3]:
# Split train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardisation
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

### 3.2 Équilibrage des classes

In [4]:
# Sur-échantillonnage (pour équilibrer les classes)
ros = RandomOverSampler(random_state=42)
print("y_train count:", y_train.value_counts())
X_train_resampled, y_train_resampled = ros.fit_resample(X_train, y_train)
print("y_train count:", y_train_resampled.value_counts())

y_train count: cluster
0    323
1    244
Name: count, dtype: int64
y_train count: cluster
0    323
1    323
Name: count, dtype: int64


## 4. Initialisation des modèles

Configuration des algorithmes de machine learning à comparer.

In [5]:
# --- Initialisation des modèles ---
models = {
    "Random Forest": RandomForestClassifier(random_state=42),
    "SVM": SVC(probability=True, random_state=42),
    "Gradient Boosting": GradientBoostingClassifier(random_state=42),
    "Régression Logistique": LogisticRegression(max_iter=1000, random_state=42)
}

## 5. Entraînement et évaluation initiale

Première évaluation des modèles avec les paramètres par défaut.

In [6]:
# --- Entraînement et évaluation initiale ---
print("=== Entraînement initial des modèles ===")
for name, model in models.items():
    model.fit(X_train_resampled, y_train_resampled)
    y_pred = model.predict(X_test)

    print(f"\nModèle : {name}")
    print(f"Accuracy    : {accuracy_score(y_test, y_pred):.4f}")
    print(f"Precision   : {precision_score(y_test, y_pred, average='macro'):.4f}")
    print(f"Recall      : {recall_score(y_test, y_pred, average='macro'):.4f}")
    print(f"F1-score    : {f1_score(y_test, y_pred, average='macro'):.4f}")
    print("Matrice de confusion :")
    print(confusion_matrix(y_test, y_pred))
    print("\nRapport complet :")
    print(classification_report(y_test, y_pred))

=== Entraînement initial des modèles ===

Modèle : Random Forest
Accuracy    : 0.9577
Precision   : 0.9588
Recall      : 0.9562
F1-score    : 0.9573
Matrice de confusion :
[[75  2]
 [ 4 61]]

Rapport complet :
              precision    recall  f1-score   support

           0       0.95      0.97      0.96        77
           1       0.97      0.94      0.95        65

    accuracy                           0.96       142
   macro avg       0.96      0.96      0.96       142
weighted avg       0.96      0.96      0.96       142


Modèle : SVM
Accuracy    : 0.9930
Precision   : 0.9924
Recall      : 0.9935
F1-score    : 0.9929
Matrice de confusion :
[[76  1]
 [ 0 65]]

Rapport complet :
              precision    recall  f1-score   support

           0       1.00      0.99      0.99        77
           1       0.98      1.00      0.99        65

    accuracy                           0.99       142
   macro avg       0.99      0.99      0.99       142
weighted avg       0.99      0.9

## 6. Validation croisée

Évaluation de la robustesse des modèles avec validation croisée 5-fold.

In [7]:
# --- Validation croisée 5-fold avec plusieurs métriques ---
print("\n=== Validation croisée 5-fold avec plusieurs métriques ===")
scoring = ['accuracy', 'precision_macro', 'recall_macro', 'f1_macro']
cv_results = {}

for name, model in models.items():
    scores = cross_validate(model, X_train_resampled, y_train_resampled, cv=5, scoring=scoring, n_jobs=-1, return_train_score=False)
    cv_results[name] = {metric: scores['test_'+metric].mean() for metric in scoring}
    print(f"\n{name} :")
    for metric in scoring:
        print(f"  {metric} moyen = {cv_results[name][metric]:.4f}")


=== Validation croisée 5-fold avec plusieurs métriques ===

Random Forest :
  accuracy moyen = 0.9629
  precision_macro moyen = 0.9637
  recall_macro moyen = 0.9628
  f1_macro moyen = 0.9628

Random Forest :
  accuracy moyen = 0.9629
  precision_macro moyen = 0.9637
  recall_macro moyen = 0.9628
  f1_macro moyen = 0.9628

SVM :
  accuracy moyen = 0.9938
  precision_macro moyen = 0.9940
  recall_macro moyen = 0.9938
  f1_macro moyen = 0.9938

SVM :
  accuracy moyen = 0.9938
  precision_macro moyen = 0.9940
  recall_macro moyen = 0.9938
  f1_macro moyen = 0.9938

Gradient Boosting :
  accuracy moyen = 0.9675
  precision_macro moyen = 0.9678
  recall_macro moyen = 0.9674
  f1_macro moyen = 0.9675

Régression Logistique :
  accuracy moyen = 0.9954
  precision_macro moyen = 0.9954
  recall_macro moyen = 0.9954
  f1_macro moyen = 0.9954

Gradient Boosting :
  accuracy moyen = 0.9675
  precision_macro moyen = 0.9678
  recall_macro moyen = 0.9674
  f1_macro moyen = 0.9675

Régression Logistiq

## 7. Optimisation des hyperparamètres

### 7.1 Définition des grilles d'hyperparamètres

In [8]:
# --- Grilles d'hyperparamètres pour GridSearchCV ---
params = {
    'Random Forest': {
        'n_estimators': [50, 100, 200],
        'max_depth': [10, 20, None]
    },
    'SVM': {
        'C': [0.1, 1, 10],
        'kernel': ['linear', 'rbf']
    },
    'Gradient Boosting': {
        'n_estimators': [50, 100, 200],
        'learning_rate': [0.01, 0.1, 0.2]
    },
    'Régression Logistique': {
        'C': [0.1, 1, 10],
        'penalty': ['l2'],
        'solver': ['lbfgs']
    }
}

### 7.2 Fonction d'optimisation et exécution

In [9]:
# --- Fonction d'optimisation avec GridSearchCV ---
def tune_models(X, y, models, params, cv=5, scoring='f1_macro'):
    best_models = {}
    for name, model in models.items():
        print(f"\nOptimisation de {name}...")
        grid = GridSearchCV(model, params[name], cv=cv, scoring=scoring, n_jobs=-1)
        grid.fit(X, y)
        print(f"Meilleurs paramètres pour {name} : {grid.best_params_}")
        print(f"Meilleur score {scoring} : {grid.best_score_:.4f}")
        best_models[name] = grid.best_estimator_
    return best_models

best_models = tune_models(X_train_resampled, y_train_resampled, models, params, cv=5, scoring='f1_macro')


Optimisation de Random Forest...
Meilleurs paramètres pour Random Forest : {'max_depth': 10, 'n_estimators': 50}
Meilleur score f1_macro : 0.9690

Optimisation de SVM...
Meilleurs paramètres pour SVM : {'C': 10, 'kernel': 'linear'}
Meilleur score f1_macro : 0.9984

Optimisation de Gradient Boosting...
Meilleurs paramètres pour Random Forest : {'max_depth': 10, 'n_estimators': 50}
Meilleur score f1_macro : 0.9690

Optimisation de SVM...
Meilleurs paramètres pour SVM : {'C': 10, 'kernel': 'linear'}
Meilleur score f1_macro : 0.9984

Optimisation de Gradient Boosting...
Meilleurs paramètres pour Gradient Boosting : {'learning_rate': 0.1, 'n_estimators': 200}
Meilleur score f1_macro : 0.9721

Optimisation de Régression Logistique...
Meilleurs paramètres pour Régression Logistique : {'C': 0.1, 'penalty': 'l2', 'solver': 'lbfgs'}
Meilleur score f1_macro : 0.9984
Meilleurs paramètres pour Gradient Boosting : {'learning_rate': 0.1, 'n_estimators': 200}
Meilleur score f1_macro : 0.9721

Optimis

## 8. Évaluation finale des modèles optimisés

Évaluation complète des modèles optimisés sur l'ensemble de test.

In [10]:
# --- Ré-évaluation des modèles optimisés sur le test set ---
print("\n=== Évaluation des modèles optimisés sur le test set ===")
final_scores = {}
for name, model in best_models.items():
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, average='macro')
    rec = recall_score(y_test, y_pred, average='macro')
    f1 = f1_score(y_test, y_pred, average='macro')
    final_scores[name] = f1

    print(f"\nModèle : {name}")
    print(f"Accuracy    : {acc:.4f}")
    print(f"Precision   : {prec:.4f}")
    print(f"Recall      : {rec:.4f}")
    print(f"F1-score    : {f1:.4f}")
    print("Matrice de confusion :")
    print(confusion_matrix(y_test, y_pred))
    print("\nRapport complet :")
    print(classification_report(y_test, y_pred))


=== Évaluation des modèles optimisés sur le test set ===

Modèle : Random Forest
Accuracy    : 0.9577
Precision   : 0.9574
Recall      : 0.9574
F1-score    : 0.9574
Matrice de confusion :
[[74  3]
 [ 3 62]]

Rapport complet :
              precision    recall  f1-score   support

           0       0.96      0.96      0.96        77
           1       0.95      0.95      0.95        65

    accuracy                           0.96       142
   macro avg       0.96      0.96      0.96       142
weighted avg       0.96      0.96      0.96       142


Modèle : SVM
Accuracy    : 0.9930
Precision   : 0.9924
Recall      : 0.9935
F1-score    : 0.9929
Matrice de confusion :
[[76  1]
 [ 0 65]]

Rapport complet :
              precision    recall  f1-score   support

           0       1.00      0.99      0.99        77
           1       0.98      1.00      0.99        65

    accuracy                           0.99       142
   macro avg       0.99      0.99      0.99       142
weighted avg   

## 9. Sélection et sauvegarde du meilleur modèle

Identification du modèle avec les meilleures performances et sauvegarde pour utilisation future.

In [11]:
# --- Sélection et sauvegarde du meilleur modèle (meilleur F1 macro) ---
best_model_name = max(final_scores, key=final_scores.get)
best_model = best_models[best_model_name]
print(f"\n🏆 Meilleur modèle final : {best_model_name} avec un F1-score macro de {final_scores[best_model_name]:.4f}")

joblib.dump(best_model, '../models/model.pkl')
print("Modèle sauvegardé dans '../models/model.pkl'")


🏆 Meilleur modèle final : SVM avec un F1-score macro de 0.9929
Modèle sauvegardé dans '../models/model.pkl'


In [12]:
# Sauvegarde du scaler pour utilisation dans l'application Streamlit
import joblib
joblib.dump(scaler, '../models/scaler.pkl')
print("Scaler sauvegardé dans '../models/scaler.pkl'")

Scaler sauvegardé dans '../models/scaler.pkl'
